# Find lots of dams using WOFLs

**What does this notebook do?** This notebook uses output `NetCDF` files of WOFS summaries from `datacube-stats` to generate polygons of water bodies in the landscape. This code follows the following workflow:
* Generate a list of netCDF files within a specified folder location
* Opens each netCDF file and:
    * Keep only pixels identified as wet at least 10% of the time
    * Convert the raster data into polygons
    * Filter the polygons based on size and proximity to identified major rivers
    * Remove any named water bodies - e.g. lakes
* Append the final polygon set to a shapefile

**Required inputs:** 
This code requires that you have run `datacube-stats` to produce WOFS summaries as netCDF files. This code takes in the folder location to generate a list of netCDF files to process.

**Date:** September 2018

**Author:** Claire Krause

In [1]:
%pylab notebook

import rasterio.features
from shapely.geometry import Polygon, shape, mapping
from shapely.ops import unary_union
import geopandas as gp
import fiona
from fiona.crs import from_epsg
import xarray as xr
import pandas as pd
import glob
import os.path

Populating the interactive namespace from numpy and matplotlib


## Set up all the parameters for the script

### How wet does a pixel need to be to be included?
The value set here will be the minimum amount of time (as a decimal between 0 and 1) that you want water to be detected before it is included in the analysis. 

E.g. If this was set to 0.25, any pixels that are wet *at least* 25% of the time will be included. If you are looking for persistant water bodies, you will want to set this threshold higher. If you don't want to use this filter, set this value to 0.

In [2]:
AtLeastThisWet = 0.10

### How big/small should the polygons be?
This filtering step can remove very small and/or very large polygons. The size listed here is in m2. A single pixel in Landsat data is 25 m X 25 m = 625 m2. 

**MinSize**

E.g. A minimum size of 6250 means that polygons need to be at least 10 pixels to be included. If you don't want to use this filter, set this value to 0.

**MaxSize**

E.g. A maximum size of 1 000 000 means that you only want to consider polygons less than 1 km2. If you don't want to use this filter, set this number to something stupidly large.

In [3]:
MinSize = 6300
MaxSize = 10000000000000000000000000000000000000000000000000000

### Do you want to filter out polygons that intersect with major rivers?

We use the [surface hydrology lines dataset](http://pid.geoscience.gov.au/dataset/ga/83107) to filter out polygons that intersect with major rivers. 
Note that we have filtered this dataset to only keep rivers tagged as `major`, and it is this filtered dataset that we use here.

If you don't want to filter out polygons that intersect with rivers, set this parameter to `False`.

In [4]:
FilterOutRivers = False

## Is this analysis for all of Australia, or just a subset?

If you would like to analyse only some part of Australia, you will need to provide a `list` of Albers tiles that cover that region. In this case, we are only analysing the Murray Darling Basin. We have previously run a custom `datacube-stats` analysis using this region, and so we can generate a list of tiles that we know covers this area using the outputs of this analysis.

In [5]:
SubRegion = True

In [6]:
# Where are the datacube-stats netCDF files located?
WOFSMDBFolder = '/g/data/r78/cek156/datacube_stats/WOFSDamsUpTo2013/'

MDBtiles = glob.glob(f'{WOFSMDBFolder}*.nc')

SubRegionAlbersTiles = set()
for filePath in MDBtiles:
    AlbersTiles = filePath.split('_')
    ThisTile = f'{AlbersTiles[3]}_{AlbersTiles[4]}'
    SubRegionAlbersTiles.add(ThisTile)
SubRegionAlbersTiles = list(SubRegionAlbersTiles)

### Set up some file names for the inputs and outputs

In [7]:
# Where are the datacube-stats netCDF files located?
WOFSInputFolder = '/g/data/u46/users/bt2744/work/data/wofs/output/prod/annualstats/'

# The name and filepath of the intermediate output polygon set
WOFSshp = '/g/data/r78/cek156/dea-notebooks/Dams/MinMaxSizeRiverFilteredDams/AllNSW201510pc.shp'

# The name and filepath of the final corrected output polygon set
WOFSshpMerged = '/g/data/r78/cek156/dea-notebooks/Dams/MinMaxSizeRiverFilteredDams/AllNSW201510pcMerged.shp'

In [8]:
# Read in the major rivers dataset (if you are using it)
if FilterOutRivers == True:
    FilteredMajorRivers = '/g/data/r78/cek156/ShapeFiles/SurfaceHydrologyLinesRegionalFilteredMAJOR.shp'
    MajorRivers = gp.GeoDataFrame.from_file(FilteredMajorRivers) 
    MajorRivers = MajorRivers.to_crs({'init':'epsg:3577'})

### Get the list of netcdf file names to loop through

If we are using the `annual WOfS` product, the `glob.glob` format will be (substituting the required year):

`glob.glob('{0}*_2014_summary.nc'.format(WOFSInputFolder))`

If we are using the custom `datacube-stats` package to look at a combination of years together, the `glob.glob` format will be:

`glob.glob('{0}*.nc'.format(WOFSInputFolder))`

In [9]:
if SubRegion == True:
    Alltiles = []
    for tile in SubRegionAlbersTiles:
        Tiles = glob.glob(f'{WOFSInputFolder}*_{tile}_2015_summary.nc')
        Alltiles.append(Tiles[0]) # Assumes only one file will be returned
else:
    Alltiles = glob.glob(f'{WOFSInputFolder}*_2015_summary.nc')

## Read in and process the data

In [10]:
for WOFSfile in Alltiles: 
    try:
        # Read in the data
        WOFSnetCDFData = xr.open_rasterio(f'NETCDF:{WOFSfile}:frequency')

        # Filter our classified data layer to remove noise
        # Remove any pixels that are wet < AtLeastThisWet% of the time
        WOFSfiltered = WOFSnetCDFData > AtLeastThisWet
        # Remove the superfluous time dimension
        WOFSfiltered = WOFSfiltered.squeeze()
        # Change all zeros to NaN
        WOFSfiltered = WOFSfiltered.where(WOFSfiltered !=0)

        # Convert the raster to polygons
        WOFSpolygons = rasterio.features.shapes(WOFSfiltered.data.astype('float32'), 
                                                transform = WOFSnetCDFData.transform[:-3])
        WOFSlist = list(WOFSpolygons)

        # Remove the nan polygons
        WOFSlist = [x for x in WOFSlist if x[1] == 1]

        # The rasterio.features.shapes returns a tuple. We only want to keep the geometry portion
        WOFSOFSbreaktuple = [a for a, b in WOFSlist]
        # Grab the geometries and convert into a shapely geometry
        for poly in WOFSOFSbreaktuple:
            poly['geometry'] = shape(poly)

        # Calculate the area of each polygon
        WOFLshapes = []
        for i, WOFLshape in enumerate(WOFSOFSbreaktuple):
            polyArea = Polygon(WOFLshape['coordinates'][0]).area
            WOFLshape['properties'] = {'area': polyArea}
            WOFLshapes.append(WOFLshape)

        # Filter out any polygons smaller than MinSize, and greater than MaxSize
        AreasIndex = [i for i, x in enumerate(WOFLshapes) if 
                      (x['properties']['area'] > MinSize) & 
                      (x['properties']['area'] < MaxSize)]
        WOFSOFSbig = [WOFLshapes[x] for x in AreasIndex]

        if FilterOutRivers:
            # Remove any polygons that intersect with a major river
            # We are only interested in farm dams, so do not need the WOFS polygons for the rivers. 
            WOFSOFSfiltered2p0 = gp.GeoDataFrame(WOFSOFSbig).set_geometry('geometry')
            Intersections= gp.sjoin(MajorRivers, WOFSOFSfiltered2p0, how="inner", op='intersects')
            IntersectIndex = sorted(list(set(Intersections['index_right'])))

            WOFSOFS = [WOFSOFSbig[x] for x in range(len(WOFSOFSbig)) 
                                              if x not in IntersectIndex]
        else:
            WOFSOFS = WOFSOFSbig

        # Merge any overlapping polygons 
        MergedPolygonsGeoms = unary_union([feature['geometry'] for feature in WOFSOFS])
        MergedPolygonsGeomsList = list(MergedPolygonsGeoms)

        # Re-calculate the area of each polygon and add it as an attribute to our new merged polygons
        #( For some reason, doing this merge prior to filtering doesn't work properly, hence calculating
        # area twice...)
        MergedPolygons = []
        for i, WOFLshape in enumerate(MergedPolygonsGeomsList):
            PolygonBits = mapping(WOFLshape)
            polyArea = Polygon(WOFLshape).area
            PolygonBits['properties'] = {'area': polyArea}
            MergedPolygons.append(PolygonBits)

        # Save the polygons to a shapefile
        schema = {'geometry': 'Polygon','properties': {'area': 'str'}}

        if os.path.isfile(WOFSshp):
            with fiona.open(WOFSshp, "a", crs = from_epsg(3577), driver = 'ESRI Shapefile', schema = schema) as output:
                for i in range(len(MergedPolygons)):
                    output.write({'properties': {'area': MergedPolygons[i]['properties']['area']},
                                'geometry': {'type': MergedPolygons[i]['type'], 'coordinates': MergedPolygons[i]['coordinates']}})
        else:
            with fiona.open(WOFSshp, "w", crs = from_epsg(3577), driver = 'ESRI Shapefile', schema = schema) as output:
                for i in range(len(MergedPolygons)):
                    output.write({'properties': {'area': MergedPolygons[i]['properties']['area']},
                                'geometry': {'type': MergedPolygons[i]['type'], 'coordinates': MergedPolygons[i]['coordinates']}})
    except:
        print(WOFSfile)

/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  w

/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  w

/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  w

/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')
/g/data/v10/public/modules/dea-env/20180728/lib/python3.6/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  w

## Merge polygons that have an edge at a tile boundary

Now that we have all of the polygons across our whole region of interest, we need to check for artifacts in the data caused by tile boundaries. 

In [11]:
# Load in a shapefile of the albers tile boundaries
# We have created a shapefile that consists of the albers tile boundaries,
# plus a 1 pixel (25 m) buffer. This shapefile will help us to find any polygons that have a
# boundary at the edge of an albers tile. We can then find where polygons touch across this
# boundary, and join them up.

AlbersBuffer = gp.read_file(
    '/g/data/r78/cek156/ShapeFiles/AlbersBuffer25m.shp')

DamPolygons = gp.read_file(WOFSshp)

# Find where the albers polygon overlaps with our dam polygons
# Perform a spatial join for polygons that intersect
Intersects = gp.sjoin(AlbersBuffer, DamPolygons, how='inner', op='intersects')
# Get the index of the dam polygons that intersect with the Albers buffer
IntersectIndex = sorted(Intersects['index_right'])

# Get just the dams that intersect with the tile boundaries
BoundaryDams = DamPolygons.iloc[IntersectIndex]
NotBoundaryDams = DamPolygons.loc[~DamPolygons.index.isin(IntersectIndex)]

# Now combine overlapping polygons in `BoundaryDams`
UnionBoundaryDams = BoundaryDams.unary_union

# `Explode` the multipolygon back out into individual polygons
UnionGDF = gp.GeoDataFrame(crs=DamPolygons.crs, geometry=[UnionBoundaryDams])
MergedDams = UnionGDF.explode()

# Then combine our new merged polygons with the `NotBoundaryDams`
# Combine New merged polygons with the remaining polygons that are not near the tile boundary
AllTogether = gp.GeoDataFrame(pd.concat([NotBoundaryDams, MergedDams],
                                        ignore_index=True, sort=True)).set_geometry('geometry')

# Calculate the area of each polygon
AllTogether['area'] = AllTogether.area

# Then write the lot out to shapefile
AllTogether.crs = {'init': 'epsg:3577'}
AllTogether.to_file(WOFSshpMerged, driver='ESRI Shapefile')

## Apply Land/Sea mask

To do.